In [1]:
# !pip install Praw
# !pip install --upgrade praw

In [2]:
#import sys
#!{sys.executable} -m pip install praw
import praw
import pandas as pd
import datetime as dt
import json

f = open("secret_python.txt", "r")
client_id = f.readline().rstrip()
secret_key = f.readline().rstrip()
f.close()

In [3]:
reddit = praw.Reddit(client_id=client_id, \
                     client_secret=secret_key, \
                     user_agent='Scrape')

reddit.read_only

True

In [4]:
dank_meme = reddit.subreddit('dank_meme')

In [5]:
memesDict = {}
memesDict["_default"] = {}
iterator = 1

In [ ]:
for submission in dank_meme.random_rising(limit = 100):
    memesDict["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "author":submission.author.name, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url}
    iterator +=1

In [ ]:
print(memesDict["_default"][2])

In [ ]:
with open('memesDatabase.json', 'w') as json_file:
  json.dump(memesDict, json_file)

In [ ]:
#Used to find out what fields does an object have
#import pprint
#pprint.pprint(vars(submission.author))